## Data preprocessing  
I will perform data preprocessing following the approach undertaken by the authors of the work 'Assessment of Ensemble-Based Machine Learning Algorithms for Exoplanet Identification' combined with my own proposition - I will mark it explicitly when the latter is applied. I will use the same dataset as they did for Kepler - not the full one, but the default one downloaded after following this link: https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=cumulative. I will then follow this approach for TESS and K2 datasets to finally obtain a marged dataset of all three of these datasets.

In [77]:
import pandas as pd 

kepler = pd.read_csv('datasets/KOI.csv', comment='#')
kepler.shape

(9564, 49)

In [78]:
kepler.isnull().sum().sort_values(ascending=False)

koi_teq_err2         9564
koi_teq_err1         9564
kepler_name          6817
koi_score            1510
koi_steff_err2        483
koi_srad_err2         468
koi_srad_err1         468
koi_slogg_err2        468
koi_slogg_err1        468
koi_steff_err1        468
koi_depth_err2        454
koi_impact_err2       454
koi_depth_err1        454
koi_duration_err2     454
koi_duration_err1     454
koi_impact_err1       454
koi_time0bk_err2      454
koi_time0bk_err1      454
koi_period_err2       454
koi_period_err1       454
koi_impact            363
koi_prad_err1         363
koi_srad              363
koi_slogg             363
koi_teq               363
koi_prad_err2         363
koi_prad              363
koi_depth             363
koi_steff             363
koi_model_snr         363
koi_tce_plnt_num      346
koi_tce_delivname     346
koi_insol             321
koi_insol_err1        321
koi_insol_err2        321
koi_kepmag              1
kepoi_name              0
koi_duration            0
koi_time0bk 

#### Step 1
Remove 7 columns: due to their lack of contribution to exoplanet identification and because of their emptiness. The authors have also dropped the column 'kepid', but I keep it since it contains duplicates - to avoid data leakage, the dataset will later be grouped by 'kepid'.

In [79]:
drop = ['kepoi_name', 'kepler_name', 'koi_pdisposition', 'koi_score', 'koi_teq_err1', 'koi_teq_err2']
kepler_clean = kepler.drop(drop, axis=1)

#### Step 1a
My proposition is to remove all columns with uncertainty (err1/err2). We might also perform further preprocessing, similar to the one in kepler.ipynb, but that is my invention. 

#### Step 2
Extract the rows where 'koi_disposition' is either CONFIRMED or FALSE POSITIVE - so remove the ones labeled as CANDIDATE. This is the only difference to what the scholars did - they dropped the FALSE POSITIVE rows. However, since CONFIRMED rows are actual planets, and FALSE POSITIVE have been examined to not be planets, I believe it's better to feed these rows to our model and drop the CANDIDATE rows, which may be either of the two and hence only spoils our model.  
Binary encoding will be performed in this code block as well - CONFIRMED will become 1 and FALSE POSITIVE will become 0 

In [80]:
kepler_candidates_df = kepler_clean[kepler_clean["koi_disposition"].str.strip().str.upper() == "CANDIDATE"].copy()

kepler_labeled_df = kepler_clean[kepler_clean["koi_disposition"].str.strip().str.upper().isin(["CONFIRMED","FALSE POSITIVE"])].copy()

kepler_labeled_df["label"] = (
    kepler_labeled_df["koi_disposition"]
    .str.strip().str.upper()
    .map({"CONFIRMED": 1, "FALSE POSITIVE": 0})
)

kepler_labeled_df.drop(["koi_disposition"], axis=1, inplace=True)

In [86]:
kepler_labeled_df.shape

(7585, 43)

#### Step 3
Impute 'koi_tce_delivname' using the author's approach.

In [81]:
dummies = pd.get_dummies(kepler_labeled_df['koi_tce_delivname'], drop_first=True, dtype=int)
kepler_labeled_imp = pd.concat([kepler_labeled_df, dummies], axis=1)
kepler_labeled_imp = kepler_labeled_imp.drop('koi_tce_delivname', axis=1)
kepler_labeled_imp

,kepid,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,label,q1_q17_dr24_tce,q1_q17_dr25_tce
0,10797460,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,1,0,1
1,10797460,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347,1,0,1
3,10848459,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597,0,0,1
4,10854555,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509,1,0,1
5,10872983,0,0,0,0,11.094321,2.036000e-05,-2.036000e-05,171.201160,0.001410,...,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9557,10205598,1,0,0,0,373.893980,1.661000e-02,-1.661000e-02,261.496800,0.034900,...,-0.052,0.699,0.074,-0.066,290.93597,47.290730,14.911,0,0,1
9558,10031643,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,...,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478,0,0,1
9559,10090151,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,...,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082,0,0,1
9561,10147276,0,0,1,0,0.681402,2.434000e-06,-2.434000e-06,132.181750,0.002850,...,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385,0,0,1


In [82]:
frac_any_nan = kepler_labeled_imp.isna().any(axis=1).mean()

n_any_nan = kepler_labeled_imp.isna().any(axis=1).sum()

n_rows = len(kepler_labeled_imp)

print(f"{n_any_nan} out of {n_rows} rows have at least one missing value " f"({frac_any_nan:.1%}).")

flags = ["koi_fpflag_nt", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec", "koi_tce_plnt_num", "q1_q17_dr24_tce", "q1_q17_dr25_tce"]
for col in flags :
    print(f"Percentage of missing values in {col}: {kepler_labeled_imp[col].isnull().sum()/kepler_labeled_imp.shape[0]}")

653 out of 7585 rows have at least one missing value (8.6%).
Percentage of missing values in koi_fpflag_nt: 0.0
Percentage of missing values in koi_fpflag_ss: 0.0
Percentage of missing values in koi_fpflag_co: 0.0
Percentage of missing values in koi_fpflag_ec: 0.0
Percentage of missing values in koi_tce_plnt_num: 0.03770599868160844
Percentage of missing values in q1_q17_dr24_tce: 0.0
Percentage of missing values in q1_q17_dr25_tce: 0.0


#### Step 4  
Numerical variables will be imputed - however, there's no sensible way to impute 'koi_tce_plnt_num', so the missing rows will be dropped.

In [83]:
kepler_labeled_final = kepler_labeled_imp.dropna(subset=['koi_tce_plnt_num'], axis=0)

#### Step 5
As shown above, 8.6% of the rows contain a missing value - therefore, these values will be imputed using the mean. To avoid data leakage, it will be done after a split to train and test sets. This is my idea - there is no mention of handling missing values aside from the column 'koi_tce_delivname' in the aforementioned work. There must be no missing values for the models to learn, so the simplest and most intuitive imputation strategy was picked. 70% of data will be split for training and 30% for testing - as proposed by the scholars.

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

drop = ["label", "kepid"]
feat_cols = [c for c in kepler_labeled_df.columns if c not in drop]
cont = ["koi_period", "koi_time0bk", "koi_impact", "koi_duration", "koi_depth", "koi_prad", "koi_teq", "koi_insol", "koi_model_snr", 
        "koi_steff", "koi_slogg", "koi_srad", "ra", "dec", "koi_kepmag", "koi_period_err1", "koi_period_err2", "koi_time0bk_err1", "koi_time0bk_err2", 
                "koi_impact_err1", "koi_impact_err2", "koi_duration_err1", "koi_duration_err2", 
                "koi_depth_err1", "koi_depth_err2", "koi_prad_err1", "koi_prad_err2", 
                "koi_insol_err1", "koi_insol_err2", "koi_steff_err1", "koi_steff_err2", 
                "koi_slogg_err1", "koi_slogg_err2", "koi_srad_err1", "koi_srad_err2"]
flags = ["koi_fpflag_nt", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec", "koi_tce_plnt_num", "q1_q17_dr24_tce", "q1_q17_dr25_tce"]

X = kepler_labeled_df[feat_cols]
y = kepler_labeled_df["label"]
groups = kepler_labeled_df["kepid"]

gss = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
groups_train = groups.iloc[train_idx]

cont_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy = "mean"), cont),
    ("scale", StandardScaler(), cont)
])

preprocess = ColumnTransformer([
    ("transform", cont_pipeline, cont),
    ("pass", "passthrough", flags)
])